## New analysis goal: fixed skill sets from tourneys

Use df_matches to identify the group ids that use competitive rulesets.
This gives us the team ids we need.
Then we can get the team roster using https://fumbbl.com/api/team/get/1102662
we need the player ids here.

and the chosen skills using https://fumbbl.com/api/team/getOptions/1102662
this is a string of player ids, with skill ids.

through https://fumbbl.com/api/skill/list we can get a list of skills.
https://fumbbl.com/api/skill/list/2020 and this is the 2020 list.

* World cup training: 9941 (dec 2020), 2 matches in dec 2020 using bb2016, then in okt 2021 for real. 429 matches.
* SUper league: 15615
* templars road to WC: 11605
* entrainment tournois: 12879
* NAF online tournaments : 9298
* Tacklezone: 12013
* Doppelbock: 13198
* Eurobowl practice league: 15643
* Eurobowl 2020 training: 12087 (eurobowl warsaw)

In [ ]:
import pandas as pd
import numpy as np
import plotnine as p9
import requests

from mizani.formatters import date_format

# point this to the location of the HDF5 datasets
path_to_datasets = 'datasets/current/'

# FUMBBL matches
target = 'df_matches.h5'
df_matches = pd.read_hdf(path_to_datasets + target) 

target = 'df_mbt.h5'
df_mbt = pd.read_hdf(path_to_datasets + target) 

In [ ]:
# FUMBBL inducements
target = 'inducements.h5'
inducements = pd.read_hdf(path_to_datasets + target) 

# FUMBBL skills
target = 'df_skills.h5'
df_skills = pd.read_hdf(path_to_datasets + target) 

In [ ]:
%run read_json_file.py
%run write_json_file.py

%run get_team_roster.py

https://fumbbl.com/api/ruleset/get/188
contains info about gold and skills.

From the roster id we can infer the bb version.

# Gather Team roster info

need a tournament or group (here called league) id.

In [ ]:
is_tournament = 1

# NAF EurOpen 2020 (1.1M, BB2016)
tournament_ids = [53038, 53037, 53040, 53041]
# NAF GBFU 2021 (1.1M, BB2020, no skills available)
tournament_ids = [56214, 56208, 56212, 56213]


# NAF Road to Malta 2022(1.15M BB2020)
tournament_ids = [58323,58324, 58322, 58321]

#is_tournament = 0

# World Cup Training (1.15M BB2020, mix of EB and WC)
#league_ids = [9941]
# Super league [BB2020, no stars, only bribes and master chef, 1.15M, bespoke tiering system]
#league_ids = [15615]



if is_tournament:
    team_ids = []

    for tournament_id in tournament_ids:
        
        tmp_list = (df_matches.query('tournament_id == @tournament_id')['team1_id'].tolist() + 
            df_matches.query('tournament_id == @tournament_id')['team2_id'].tolist())
            
        tmp_list = list(set(tmp_list))

        team_ids.extend(tmp_list)
else:
    team_ids = []

    for league_id in league_ids:
        
        tmp_list = (df_matches.query('league == @league_id')['team1_id'].tolist() + 
            df_matches.query('league == @league_id')['team2_id'].tolist())
            
        tmp_list = list(set(tmp_list))

        team_ids.extend(tmp_list)

In [ ]:
# how many teams?
len(team_ids)
#46*4

The `get_team_roster()` function is the workhorse here, piecing together the roster containing skills and inducements. For this it needs match data as inducements are considered part of a match, not of a roster. 

In [ ]:
df_rosters = get_team_roster(team_ids[0], df_skills, df_matches, inducements)

team_ids.pop(0)

for team_id in team_ids:
    df_rosters_tmp = get_team_roster(team_id, df_skills, df_matches, inducements)
    df_rosters = pd.concat([df_rosters, df_rosters_tmp], ignore_index=True)

In [ ]:
df_rosters

In [ ]:
#target = 'datasets/current/df_rosters_eurobowl2020'
target = 'datasets/current/df_rosters_road_to_malta'
#target = 'datasets/current/df_rosters_super_league'
#target = 'datasets/current/df_rosters_wc_training'

#df_rosters.to_hdf(target + '.h5', key='df_rosters', mode='w', format = 't',  complevel = 9)
df_rosters.to_csv(target + '.csv')

